In [ ]:
# https://www.tensorflow.org/lite/tutorials/model_maker_image_classification
from __future__ import absolute_import, division
from BashColors import C
from CV2_Utils_2 import *
from TarfileFunctions import *


from time import perf_counter, sleep
import itertools, os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'

import numpy as np
import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

contentPath=os.getcwd()
image_path=os.path.join(contentPath, 'images')
generatorPath=os.path.join(contentPath, 'DataGenerator')
generatorPath

In [ ]:
data = DataLoader.from_folder(generatorPath)
train_ds, val_ds = data.split(0.8)
print()
print('\nnum_classes:', train_ds.num_classes)
print('class names', train_ds.index_to_label)
print(len(train_ds) // 8, len(val_ds)//8)

In [ ]:
from tflite_model_maker import image_classifier
BATCH_SIZE = 8
EPOCHS = 1
DROPOUT_RATE = 0.5
LEARNING_RATE = 0.001

model = image_classifier.create(
    train_data = train_ds,
    validation_data = val_ds,
    model_spec = model_spec.get('mobilenet_v2'),
    learning_rate = LEARNING_RATE,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    dropout_rate = DROPOUT_RATE,
    use_augmentation = True, # default=False
    use_hub_library = True,  # default=True
    momentum = 0.9, # Only used when use_hub_library is True
    shuffle = False,
    train_whole_model = False
)

In [ ]:
bs='''
model  =  tflite_model_maker.image_classifier.create(
    train_ds, model_spec = 'mobilenet_v2',
    validation_data = val_ds, # default
    batch_size = None, # default None
    epochs = None,  # default None
    steps_per_epoch = None,  # default None
    train_whole_model = None, # default None
    dropout_rate = None,  # default None
    learning_rate = None,  # default None
    momentum = None, # default None
    shuffle = False,
    use_augmentation = False, # default False
    use_hub_library = True, warmup_steps = None,  # default None
    model_dir = None, # default None
    do_train = True
)
'''

In [ ]:
q

In [ ]:
model.ALLOWED_EXPORT_FORMAT

In [ ]:
loss, accuracy = model.evaluate(data=val_ds)

In [ ]:
model.export(export_dir='.')

In [ ]:
# help(DeprecationWarning)

In [ ]:
testData = DataLoader.from_folder(image_path, shuffle=False)
print('\n', type(testData), '\n')
# help(DataLoader.from_folder)

In [ ]:
train_ds, rest_data = data.split(0.5)
validation_data, val_ds = rest_data.split(0.5)

In [ ]:

from tensorflow_examples.lite.model_maker.core.data_util.image_dataloader import ImageClassifierDataLoader
def plotDatasetImages(dataset:ImageClassifierDataLoader):
    """ """
    plt.figure(figsize=(12,12))
    for i, (image, label) in enumerate(dataset.gen_dataset().unbatch().take(25)):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(image.numpy(), cmap=plt.cm.gray)
        plt.xlabel(data.index_to_label[label.numpy()])
    plt.show()

plotDatasetImages(dataset=data)

In [ ]:
import tensorflow_hub as hub
module_spec = hub.load_module_spec("path/to/module")
height, width = hub.get_expected_image_size(module_spec)
images = ...  # A batch of images with shape [batch_size, height, width, 3].
module = hub.Module(module_spec)
features = module(images)   # A batch with shape [batch_size, num_features].